In [1]:
import sys

# add path to core
sys.path.append("../../core/")

from parse import *
from connectivity import *
from plot_tools import *
from physical_block import *
from ode_system import ODESystem
from bdf import BDF1, BDF2
from bcmanager import BCManager
import matplotlib.pyplot as plt
from rc_calculator import RCCalculator
from scipy.integrate import simps
from output_writer import OutputWriter


class ProblemData:
    def __init__(self):
        # tolerance to determine if two points are the same
        self.tol = 0.5
        # maxlength of the singe vessel portion
        self.maxlength = 4 * self.tol
        # density of blood
        self.density = 1.06
        # viscosity of blood
        self.viscosity = 0.04
        # elastic modulus
        self.E = 2 * 10**5
        # vessel thickness ration w.r.t. diameter
        self.thickness_ratio = 0.08
        # use pressure at inlet
        self.use_inlet_pressure = True
        # timestep size
        self.deltat = 0.005
        # initial time
        self.t0 = 0.0
        # final time
        self.T = 10
        # ramp rime
        self.t0ramp = -2.0
        # self length units of the geometry files
        self.units = "cm"


def main():
    pd = ProblemData()
    coronary = "right"
    fdr = "./"
    paths = parse_vessels(fdr, pd)
    plot_vessel_portions(paths)
    plot_show()
    chunks, bifurcations, connectivity = build_slices(paths, pd.tol, pd.maxlength)
    coeff_resistance = 0.98
    coeff_capacitance = 0.6
    rc = RCCalculator(fdr, coronary, coeff_resistance, coeff_capacitance)
    rc.assign_resistances_to_outlets(chunks, connectivity)
    rc.assign_capacitances_to_outlets(chunks, connectivity)
    blocks = create_physical_blocks(chunks, model_type = 'Windkessel2', problem_data = pd)
    bcmanager = BCManager(chunks, connectivity,
                          inletbc_type = "pressure",
                          outletbc_type = "coronary",
                          folder = fdr,
                          problem_data = pd,
                          coronary = coronary,
                          distal_pressure_coeff = 1.05,
                          distal_pressure_shift = 10)
    ode_system = ODESystem(blocks, connectivity, bcmanager)
    bdf = BDF2(ode_system, connectivity, pd, bcmanager)
    solutions, times = bdf.run()
    show_inlet_flow_vs_pressure(solutions, times, 5, 6, 0)
    show_animation(solutions, times, pd.t0, chunks, 'Q', resample = 4,
                   inlet_index = bcmanager.inletindex)

    show_inlet_vs_distal_pressure(bcmanager, 0, 1)

    # check total flow / min
    positive_times = np.where(times > pd.t0)[0]
    Pin = solutions[bcmanager.inletindex * 3 + 0, positive_times]
    Qin = solutions[bcmanager.inletindex * 3 + 2, positive_times]
    print("Flow = " + str(simps(Qin, times[positive_times]) / (pd.T - pd.t0)) + " [mL/min]")
    print("Mean inlet pressure = " + str(simps(Pin, times[positive_times]) / 1333.2 / (pd.T - pd.t0)) + " [mmHg]")
    ow = OutputWriter("output", bcmanager, chunks, pd)
    ow.write_outlet_rc()
    ow.write_distal_pressure()
    ow.write_inlet_pressure()
    plot_show()

if __name__ == "__main__":
    main()

c1.pth
c2.pth
c3.pth
c6.pth
c4.pth
c5.pth
RCA.pth


TypeError: object of type 'iterator' has no len()

<Figure size 432x288 with 1 Axes>

TypeError: only size-1 arrays can be converted to Python scalars

In [4]:
pd = ProblemData()
coronary = "right"
fdr = "./"
paths = parse_vessels(fdr, pd)

c1.pth
c2.pth
c3.pth
c6.pth
c4.pth
c5.pth
RCA.pth


In [7]:
paths[0].coords

array([[ 16.02460196,  18.7760028 , -17.21970062],
       [ 16.04307226,  18.77278945, -17.23117267],
       [ 16.09132745,  18.76355885, -17.26113435],
       [ 16.1586339 ,  18.74892518, -17.302904  ],
       [ 16.23425802,  18.72950263, -17.34979993],
       [ 16.30746619,  18.70590536, -17.39514047],
       [ 16.36961882,  18.67874067, -17.4335489 ],
       [ 16.42045229,  18.64858834, -17.46486829],
       [ 16.461797  ,  18.61602123, -17.49024667],
       [ 16.49548335,  18.58161222, -17.51083208],
       [ 16.52334173,  18.5459342 , -17.52777255],
       [ 16.54708908,  18.50929632, -17.54191032],
       [ 16.56798851,  18.47095283, -17.55286453],
       [ 16.58718967,  18.42989424, -17.55994852],
       [ 16.60584222,  18.38511108, -17.56247563],
       [ 16.62509582,  18.33559387, -17.5597592 ],
       [ 16.64542767,  18.28075483, -17.5516342 ],
       [ 16.66462524,  18.22169291, -17.54002199],
       [ 16.67980353,  18.15992874, -17.52736559],
       [ 16.68807756,  18.09698